In [ ]:
import cv2
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split
from scipy.fftpack import fft2, fftshift
from skimage.feature import greycomatrix, greycoprops

def load_image(path):
    return cv2.imread(path, cv2.IMREAD_GRAYSCALE)

def fourier_transform(image):
    f = np.fft.fft2(image)
    fshift = np.fft.fftshift(f)
    magnitude_spectrum = 20*np.log(np.abs(fshift))
    return magnitude_spectrum

def noise_features(image):
    # modelo de ruido básico
    mean_noise = np.mean(image)
    std_noise = np.std(image)
    return mean_noise, std_noise

def edge_detection(image):
    edges = cv2.Canny(image, 100, 200)
    return edges

def texture_features(image):
    g = greycomatrix(image, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4], levels=256)
    contrast = greycoprops(g, 'contrast')
    return np.mean(contrast)

def segment_image(image, k=4):
    # Transforma la imagen a un formato adecuado para k-means
    Z = image.reshape((-1,3))

    # Convierte a float32
    Z = np.float32(Z)

    # Define los criterios, número de clusters(K) y aplica k-means()
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    ret, label, center = cv2.kmeans(Z, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

    # convertir de nuevo en uint8, y que la imagen original tenga el color del cluster
    center = np.uint8(center)
    res = center[label.flatten()]
    res2 = res.reshape((image.shape))

    # Para un mejor análisis se podría procesar la imagen segmentada para extraer características adicionales
    return res2


def extract_features(image):
    ft = fourier_transform(image)
    nf = noise_features(image)
    ed = edge_detection(image)
    tf = texture_features(image)
    seg = segment_image(image) 
    return np.hstack([ft.ravel(), nf, ed.ravel(), tf.ravel(), seg.ravel()])

def prepare_dataset(images):
    features = []
    labels = []  # llenar con 0 (no manipulado) y 1 (manipulado)
    for image in images:
        feat = extract_features(image)
        features.append(feat)
        # labels.append(...) poner según corresponda
    return np.array(features), np.array(labels)

# Cargar imágenes
images = [load_image('path_to_image.jpg') for path in paths] # leer del dataset

# Preparar conjunto de datos
X, y = prepare_dataset(images)

# Entrenar modelo
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
model = svm.SVC()
model.fit(X_train, y_train)

# Evaluar modelo
accuracy = model.score(X_test, y_test)
print(f"Accuracy: {accuracy}")
